<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/%EA%B3%BC%EC%A0%95/%EC%A6%9D%EB%B6%84%EB%90%9C_m_%EB%8D%B0%EC%9D%B4%ED%84%B0_%ED%95%99%EC%8A%B5%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

     |████████████████████████████████| 47.3 MB 1.4 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 14.7 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595738 sha256=6154a2e3a68ae8d7ef7c083296578116f3646c7836f21d9c689a588ac999dc8f
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 15.0 MB/s 
     |████████████████████████████████| 769 kB 14.4 MB/s 
     |████████████████████████████████| 895 kB 76.4 MB/s 
     |████████████████████████████████| 3.0 MB 61.0 MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-sberuvkx
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/df_M_inc.csv')

## 데이터 전처리

In [27]:
df['digit_3'].value_counts()

733    9340
732    6972
739    6708
729    6540
701    6152
721    5513
716    5406
702    5100
714    4947
731    4196
711    3963
715    3318
712    3128
713    3025
Name: digit_3, dtype: int64

In [28]:
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000068,M,72,721,현장에서 토목 공사
1,id_0000440,M,72,721,"사무실에서 계약의뢰에 의해 건축설계, 감리"
2,id_0000562,M,72,721,건축사 사무소에서 고객의 의뢰에 따라 건축설계
3,id_0000910,M,72,721,사무실에서 관련 업체 요구에 의해 토지개발 예산 산출 컨설팅
4,id_0000934,M,72,721,사무실에서 관련 업체 요구에 의해 환경영향평가 보고서 제공
...,...,...,...,...,...
74303,pid_0998381,M,71,714,"의약품을 가지고 영업하는 업체를 대상 시장조사, 여론조사 마케팅 영역"
74304,pid_0998381,M,71,714,가지고 영업하는 대상 여론조사 마케팅 영역
74305,pid_0998381,M,71,714,"의약품을 가지고 영업하는 업체를 대상 시장조사, 여론조사 마케팅 영역"
74306,pid_0998381,M,71,714,"의약품을 가지고 영업하는 업체를 대상 시장조사, 여론조사 마케팅 영역"


In [29]:
df.loc[(df['digit_3'] == 701), 'digit_3'] = 0  
df.loc[(df['digit_3'] == 702), 'digit_3'] = 1  
df.loc[(df['digit_3'] == 711), 'digit_3'] = 2  
df.loc[(df['digit_3'] == 712), 'digit_3'] = 3
df.loc[(df['digit_3'] == 713), 'digit_3'] = 4  
df.loc[(df['digit_3'] == 714), 'digit_3'] = 5  
df.loc[(df['digit_3'] == 715), 'digit_3'] = 6  
df.loc[(df['digit_3'] == 716), 'digit_3'] = 7  
df.loc[(df['digit_3'] == 721), 'digit_3'] = 8  
df.loc[(df['digit_3'] == 729), 'digit_3'] = 9  
df.loc[(df['digit_3'] == 731), 'digit_3'] = 10  
df.loc[(df['digit_3'] == 732), 'digit_3'] = 11  
df.loc[(df['digit_3'] == 733), 'digit_3'] = 12  
df.loc[(df['digit_3'] == 739), 'digit_3'] = 13  

In [30]:
data_list = []
for q, label in zip(df['text'], df['digit_3'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

print(data_list[:5])

[['현장에서 토목 공사', '8'], ['사무실에서 계약의뢰에 의해 건축설계, 감리', '8'], ['건축사 사무소에서 고객의 의뢰에 따라 건축설계', '8'], ['사무실에서 관련 업체 요구에 의해 토지개발 예산 산출 컨설팅', '8'], ['사무실에서 관련 업체 요구에 의해 환경영향평가 보고서 제공', '8']]


## Train data & test data

In [31]:
from sklearn.model_selection import train_test_split             
dataset_train, dataset_test = train_test_split(data_list, test_size=0.1, random_state=0) #추후에 test_size 수정 필요

# KoBERT 입력 데이터로 만들기

In [32]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [33]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5 #추후 num_epochs 증가할 필요 있음
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

 ### 추후 과제 : 쓰이는 parameters 최적화 필요

In [34]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [35]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [36]:
data_train[0]

(array([   2, 2600, 6903, 3623, 6291, 2724,  887, 6566, 5436, 2724,    3,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(11, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 8)

In [37]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

# KoBERT 학습 모델 만들기

In [38]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=14,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [39]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


# KoBERT 모델 학습시키기

In [40]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/1045 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.762700319290161 train acc 0.078125
epoch 1 batch id 201 loss 1.197884202003479 train acc 0.3931902985074627
epoch 1 batch id 401 loss 0.3173338770866394 train acc 0.647482855361596
epoch 1 batch id 601 loss 0.1476679891347885 train acc 0.7395486688851913
epoch 1 batch id 801 loss 0.21629278361797333 train acc 0.787628745318352
epoch 1 batch id 1001 loss 0.3158208131790161 train acc 0.8179476773226774
epoch 1 train acc 0.8231123519491724


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/117 [00:00<?, ?it/s]

...

epoch 4 test acc 0.969551282051282


  0%|          | 0/1045 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.16201893985271454 train acc 0.96875
epoch 5 batch id 201 loss 0.02049817517399788 train acc 0.9832866915422885
epoch 5 batch id 401 loss 0.03485608100891113 train acc 0.9840243142144638
epoch 5 batch id 601 loss 0.005511072929948568 train acc 0.9841150166389351
epoch 5 batch id 801 loss 0.1243981420993805 train acc 0.984414013732834
epoch 5 batch id 1001 loss 0.09137218445539474 train acc 0.9844218281718282
epoch 5 train acc 0.9844938818730881


  0%|          | 0/117 [00:00<?, ?it/s]

epoch 5 test acc 0.9708867521367521


In [41]:
#학습 모델 저장
torch.save(model, 'M_inc_model.pt')